# Прогнозируем задержки самолетов

In [2]:
!pip install catboost lightgbm optuna -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 3.8 MB/s eta 0:00:00


In [3]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import roc_auc_score
import pandas as pd

from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [4]:
RANDOM_STATE = 111
DATASET_PATH = 'https://raw.githubusercontent.com/evgpat/edu_stepik_practical_ml/main/datasets/flight_delays_train.csv'

In [5]:
data = pd.read_csv(DATASET_PATH)

X = data.drop('dep_delayed_15min', axis=1)
y = data['dep_delayed_15min'] == 'Y'

X.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732
1,c-4,c-20,c-3,1548,US,PIT,MCO,834
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423


In [6]:
X.dtypes

,0
Month,object
DayofMonth,object
DayOfWeek,object
DepTime,int64
UniqueCarrier,object
Origin,object
Dest,object
Distance,int64


Создайте список номеров колонок с категориальными признаками для бустингов

## Quiz
Какой длины получился список?

(подсказка: колонка `DepTime` числовая)

In [ ]:
import numpy as np

In [9]:


categorical_features_indices = np.where(X.dtypes != int)[0] # your code here
len(categorical_features_indices)

6

In [10]:
cat_features = X.select_dtypes(include=['object']).columns.to_list()
print(len(cat_features))

6


In [11]:
cat_features

['Month', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier', 'Origin', 'Dest']

Разобъем данные на обучение и контроль

In [12]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.25, random_state=RANDOM_STATE)

In [13]:
Xtrain.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
41207,c-4,c-18,c-1,1457,CO,EWR,TPA,998
28283,c-11,c-1,c-2,1225,UA,DEN,BOS,1754
34619,c-6,c-16,c-5,1650,YV,IAD,CAE,401
8789,c-5,c-18,c-4,923,AA,SLC,DFW,988
38315,c-2,c-14,c-2,1839,AA,STL,SAN,1558


## Модели с параметрами по умолчанию

Обучите CatBoost с гиперпараметрами по умолчанию.

## Quiz
Чему равен ROC-AUC на тестовых данных? Ответ округлите до сотых.

In [ ]:
from catboost import Pool, metrics, cv
from sklearn.metrics import accuracy_score, roc_auc_score

In [14]:
# your code here
model_CBC = CatBoostClassifier(
    #custom_loss=[metrics.AUC()],
    random_seed=RANDOM_STATE,
    logging_level='Silent'
)

In [15]:
model_CBC.fit(
    Xtrain, ytrain,
    cat_features=categorical_features_indices,
    #eval_set=(X_test, y_test),
    logging_level='Verbose',
    plot=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.065101
0:	learn: 0.6600024	total: 152ms	remaining: 2m 31s
1:	learn: 0.6301097	total: 274ms	remaining: 2m 16s
2:	learn: 0.6062540	total: 351ms	remaining: 1m 56s
3:	learn: 0.5861899	total: 437ms	remaining: 1m 48s
4:	learn: 0.5692165	total: 491ms	remaining: 1m 37s
5:	learn: 0.5538774	total: 549ms	remaining: 1m 31s
6:	learn: 0.5397378	total: 658ms	remaining: 1m 33s
7:	learn: 0.5274410	total: 773ms	remaining: 1m 35s
8:	learn: 0.5173510	total: 885ms	remaining: 1m 37s
9:	learn: 0.5085494	total: 987ms	remaining: 1m 37s
10:	learn: 0.5015010	total: 1.09s	remaining: 1m 37s
11:	learn: 0.4959284	total: 1.14s	remaining: 1m 33s
12:	learn: 0.4894959	total: 1.26s	remaining: 1m 36s
13:	learn: 0.4840668	total: 1.38s	remaining: 1m 37s
14:	learn: 0.4808529	total: 1.42s	remaining: 1m 33s
15:	learn: 0.4760575	total: 1.55s	remaining: 1m 35s
16:	learn: 0.4733882	total: 1.62s	remaining: 1m 33s
17:	learn: 0.4693399	total: 1.75s	remaining: 1m 35s
18:	learn: 0.4663194	total: 1.94s	remaining:

In [18]:
from catboost import Pool, metrics, cv
from sklearn.metrics import accuracy_score, roc_auc_score

In [20]:
cv_params = model_CBC.get_params()
cv_params.update({
    'loss_function':metrics.Logloss()
})
cv_data = cv(
    Pool(X, y, cat_features=categorical_features_indices),
    cv_params,
    #plot=True
)

In [22]:
cv_data

,iterations,test-Logloss-mean,test-Logloss-std,train-Logloss-mean,train-Logloss-std
0,0,0.676997,0.000017,0.676995,0.000063
1,1,0.662174,0.000230,0.662161,0.000250
2,2,0.648690,0.000278,0.648679,0.000295
3,3,0.636218,0.000459,0.636206,0.000415
4,4,0.623964,0.000918,0.623901,0.001034
...,...,...,...,...,...
995,995,0.412461,0.001223,0.403164,0.001038
996,996,0.412458,0.001216,0.403145,0.001039
997,997,0.412445,0.001227,0.403131,0.001034
998,998,0.412445,0.001229,0.403110,0.001032


In [28]:
predictions = model_CBC.predict(Xtest)
predictions_probs = model_CBC.predict_proba(Xtest)[:,1]
print(predictions[:10])
print(predictions_probs[:10])

[False False False False False False False False False False]
[0.16876749 0.07501404 0.42402944 0.03950113 0.27438622 0.0897944
 0.13123688 0.24626527 0.21237737 0.07231455]


In [30]:
round(roc_auc_score(ytest, predictions_probs), 2)

0.77

In [31]:
# ну или вообще не заморачиваясь ....

%%time

from catboost import CatBoostClassifier, Pool, metrics, cv
from sklearn.metrics import roc_auc_score

model = CatBoostClassifier(random_seed=RANDOM_STATE, verbose=0)
model.fit(Xtrain, ytrain, cat_features=categorical_features_indices)

y_pred=model.predict(Xtest)
y_pred_prob=model.predict_proba(Xtest)


roc_auc=roc_auc_score(ytest, y_pred_prob[:,1])

print(round(roc_auc, 2))

0.77
CPU times: user 3min 38s, sys: 3.93 s, total: 3min 42s
Wall time: 2min 15s


Обучите LightGBM с гиперпараметрами по умолчанию.

## Quiz
Чему равен ROC-AUC на тестовых данных? Ответ округлите до сотых.

In [32]:
from google.colab import output
output.enable_custom_widget_manager()

Support for third party widgets will remain active for the duration of the session. To disable support:

In [33]:
from google.colab import output
output.disable_custom_widget_manager()

In [34]:
for c in X.columns:
    col_type = X[c].dtype
    if col_type == 'object' or col_type.name == 'category':
        Xtrain[c] = Xtrain[c].astype('category')
        Xtest[c] = Xtest[c].astype('category')

In [35]:
# your code here

lgbm = LGBMClassifier(random_seed=RANDOM_STATE)
lgbm.fit(Xtrain, ytrain)

y_pred_lgbm=lgbm.predict(Xtest)
y_pred_prob_lgbm=lgbm.predict_proba(Xtest)


roc_auc_lgbm=roc_auc_score(ytest, y_pred_prob_lgbm[:,1])

print(round(roc_auc_lgbm, 2))

[LightGBM] [Info] Number of positive: 14346, number of negative: 60654
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003263 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1095
[LightGBM] [Info] Number of data points in the train set: 75000, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.191280 -> initscore=-1.441714
[LightGBM] [Info] Start training from score -1.441714
0.73


## Optuna

Выделим дополнительную валидационную выборку.

In [36]:
Xtrain_new, Xval, ytrain_new, yval = train_test_split(Xtrain, ytrain, test_size=0.25, random_state=RANDOM_STATE)

Создайте функцию objective_lgbm, в которой среди гиперпараметров

* num_leaves = trial.suggest_int("num_leaves", 10, 100)
* n_estimators = trial.suggest_int("n_estimators", 10, 1000)

подберите оптимальные, обучая LGBM на Xtrain_new, ytrain_new и проверяя качество (ROC-AUC) на Xval.

Используйте 30 эпох обучения Optuna.


In [37]:
import optuna

In [39]:
# your code here
def objective_lgbm(trial):

    num_leaves = trial.suggest_int("num_leaves", 10, 100)
    n_estimators = trial.suggest_int("n_estimators", 10, 1000)

    score = cross_val_score(LGBMClassifier(num_leaves=num_leaves, n_estimators=n_estimators),
                            Xtrain_new, ytrain_new, cv=3, scoring='roc_auc', n_jobs=-1).mean()
    return score


study = optuna.create_study(direction="maximize")
study.optimize(objective_lgbm, n_trials=30)

[I 2024-09-25 08:41:49,927] A new study created in memory with name: no-name-83938564-55a4-4ae6-97f3-e67d932c36b1
[I 2024-09-25 08:41:57,161] Trial 0 finished with value: 0.7104088222883246 and parameters: {'num_leaves': 76, 'n_estimators': 91}. Best is trial 0 with value: 0.7104088222883246.
[I 2024-09-25 08:42:03,418] Trial 1 finished with value: 0.7101244407099171 and parameters: {'num_leaves': 40, 'n_estimators': 258}. Best is trial 0 with value: 0.7104088222883246.
[I 2024-09-25 08:42:25,053] Trial 2 finished with value: 0.7055143830636039 and parameters: {'num_leaves': 90, 'n_estimators': 651}. Best is trial 0 with value: 0.7104088222883246.
[I 2024-09-25 08:42:40,959] Trial 3 finished with value: 0.7048718567539938 and parameters: {'num_leaves': 75, 'n_estimators': 530}. Best is trial 0 with value: 0.7104088222883246.
[I 2024-09-25 08:42:48,576] Trial 4 finished with value: 0.7070961260233792 and parameters: {'num_leaves': 84, 'n_estimators': 227}. Best is trial 0 with value: 0.

In [40]:
study.best_params

{'num_leaves': 59, 'n_estimators': 21}

In [41]:
def objective_lgbm_AUC(trial):
    # num_leaves (int, optional (default=31)) – Maximum tree leaves for base learners.
    num_leaves = trial.suggest_int("num_leaves", 10, 100)
    # n_estimators (int, optional (default=100)) – Number of boosted trees to fit.
    n_estimators = trial.suggest_int("n_estimators", 10, 1000)

    model=LGBMClassifier(num_leaves=num_leaves, n_estimators=n_estimators, random_state=RANDOM_STATE)
    model.fit(Xtrain_new, ytrain_new)

    # y_pred=model.predict(Xval)
    y_pred_prob=model.predict_proba(Xval)
    roc_auc=roc_auc_score(yval, y_pred_prob[:,1])
    return roc_auc

In [42]:
# storage = optuna.storages.InMemoryStorage()
study = optuna.create_study(direction='maximize')
study.optimize(objective_lgbm_AUC, n_trials=30)

[I 2024-09-25 08:52:34,870] A new study created in memory with name: no-name-7eff3420-418f-48a2-a8f5-4b0ace070893


[LightGBM] [Info] Number of positive: 10730, number of negative: 45520
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001432 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1095
[LightGBM] [Info] Number of data points in the train set: 56250, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.190756 -> initscore=-1.445108
[LightGBM] [Info] Start training from score -1.445108


[I 2024-09-25 08:52:36,485] Trial 0 finished with value: 0.722173491294875 and parameters: {'num_leaves': 69, 'n_estimators': 131}. Best is trial 0 with value: 0.722173491294875.


[LightGBM] [Info] Number of positive: 10730, number of negative: 45520
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001542 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1095
[LightGBM] [Info] Number of data points in the train set: 56250, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.190756 -> initscore=-1.445108
[LightGBM] [Info] Start training from score -1.445108


[I 2024-09-25 08:52:41,713] Trial 1 finished with value: 0.7157910900089 and parameters: {'num_leaves': 36, 'n_estimators': 711}. Best is trial 0 with value: 0.722173491294875.


...

[I 2024-09-25 08:54:19,758] Trial 27 finished with value: 0.7188861546292646 and parameters: {'num_leaves': 57, 'n_estimators': 403}. Best is trial 11 with value: 0.7254682469350497.


[LightGBM] [Info] Number of positive: 10730, number of negative: 45520
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002728 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1095
[LightGBM] [Info] Number of data points in the train set: 56250, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.190756 -> initscore=-1.445108
[LightGBM] [Info] Start training from score -1.445108


[I 2024-09-25 08:54:22,928] Trial 28 finished with value: 0.7234110530002772 and parameters: {'num_leaves': 86, 'n_estimators': 177}. Best is trial 11 with value: 0.7254682469350497.


[LightGBM] [Info] Number of positive: 10730, number of negative: 45520
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005610 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1095
[LightGBM] [Info] Number of data points in the train set: 56250, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.190756 -> initscore=-1.445108
[LightGBM] [Info] Start training from score -1.445108


[I 2024-09-25 08:54:25,334] Trial 29 finished with value: 0.7221974659121875 and parameters: {'num_leaves': 69, 'n_estimators': 132}. Best is trial 11 with value: 0.7254682469350497.


In [43]:
study.best_params

{'num_leaves': 73, 'n_estimators': 47}

Обучите модель с найденными гиперпараметрами на Xtrain, ytrain и оцените ROC-AUC на тестовых данных.

In [45]:
# your code here
model_lgbmc = LGBMClassifier(**study.best_params)
model_lgbmc.fit(Xtrain, ytrain)

pred_lbbmc = model_lgbmc.predict_proba(Xtest)

roc_auc=roc_auc_score(ytest, pred_lbbmc[:,1])

print(roc_auc)


[LightGBM] [Info] Number of positive: 14346, number of negative: 60654
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001940 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1095
[LightGBM] [Info] Number of data points in the train set: 75000, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.191280 -> initscore=-1.441714
[LightGBM] [Info] Start training from score -1.441714
0.7331470298702449


## Quiz

Чему равно количество деревьев в LGBM после подбора гиперпараметров?

## Работа над улучшением модели

* Попробуйте при помощи Optuna подобрать и другие гиперпарамтеры
* Также подберите гиперпараметры у CatBoost (а не только у LightGBM)

In [ ]:
# your code here

## Quiz

Поделитесь своими результатами!